In [1]:
import os
import numpy as np

%matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.image import NonUniformImage

from scipy.interpolate import interp1d


Using matplotlib backend: Qt4Agg


In [2]:
base_path = '/home/stuart/GitHub/Thesis/thesis/Chapter4/Data/'

In [3]:
def get_filepath(base_path, driver, period, post_amp, tube_r, exp_fac):
    if exp_fac is not None:
        data_dir = os.path.join(base_path, '%s/%s_%s_%s_%s/'%(driver, period, post_amp, tube_r, exp_fac))
    else:
        data_dir = os.path.join(base_path, '%s/%s_%s_%s/'%(driver, period, post_amp, tube_r))
    
    return data_dir

In [4]:
def get_data(base_path, driver, tube_r, exp_fac):

    post_amp = "A10"
    period = "p240"

    data_dir = get_filepath(base_path, driver, period, post_amp, tube_r, exp_fac)

    def path_join(filename):
        return os.path.join(data_dir, filename)

    all_svphi = np.load(path_join("LineVar_%s_%s_%s_vphi.npy"%(driver,period,post_amp))).T
    all_svperp = np.load(path_join("LineVar_%s_%s_%s_vperp.npy"%(driver,period,post_amp))).T
    all_svpar = np.load(path_join("LineVar_%s_%s_%s_vpar.npy"%(driver,period,post_amp))).T
    all_spoints = np.load(path_join("LineVar_%s_%s_%s_points.npy"%(driver,period,post_amp)))[:,::-1,:]
    all_times = np.load(path_join("LineVar_%s_%s_%s_times.npy"%(driver,period,post_amp)))[:,0]

    beta_line = np.load(path_join("LineFlux_%s_%s_%s_beta.npy"%(driver,period,post_amp))).T
    height_Mm = np.load(os.path.join(base_path, "heightMM.npy"))
    f = interp1d(np.linspace(0,128,128),height_Mm)
    y = f(all_spoints[0,:,2])
    
    if post_amp in ['A02k', 'A10']:
        data = [all_svpar * 1e3, all_svperp * 1e3, all_svphi * 1e3]
    else:
        data = [all_svpar, all_svperp, all_svphi]

    return y, data, beta_line, all_times, all_spoints


In [12]:
post_amp = "A10"
period = "p240"

In [13]:
def path_join(filename):
    return os.path.join(data_dir, filename)

In [15]:
data_dir = get_filepath(base_path, driver, period, post_amp, tube_r, exp_fac)

In [18]:
all_svphi = np.load(path_join("LineVar_%s_%s_%s_bphi.npy"%(driver,period,post_amp))).T

IOError: [Errno 2] No such file or directory: '/home/stuart/GitHub/Thesis/thesis/Chapter4/Data/horiz/p240_A10_r30/LineVar_horiz_p240_A10_bphi.npy'

In [17]:
all_svphi

array([[ -5.46994655e-294,  -8.03036395e-202,   7.58764516e-152, ...,
         -1.22777400e-002,  -1.22348975e-002,  -1.21560936e-002],
       [ -1.50449953e-288,  -6.28780106e-198,   2.00777923e-149, ...,
         -1.19903450e-002,  -1.18946192e-002,  -1.18059467e-002],
       [ -1.97833238e-283,  -2.75358305e-194,   4.24935998e-147, ...,
         -1.28739293e-002,  -1.27011008e-002,  -1.26034305e-002],
       ..., 
       [  1.25299805e-008,   3.14171138e-008,  -1.46725509e-008, ...,
          1.10293728e-002,   1.16341058e-002,   1.19855610e-002],
       [  2.28042904e-009,   5.83719902e-009,  -1.14978963e-009, ...,
         -2.35245680e-002,  -2.33142927e-002,  -2.31627891e-002],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
         -8.92190977e-003,  -9.03787234e-003,  -9.09364161e-003]])

In [5]:
def get_speeds(base_path, driver, tube_r, exp_fac):

    post_amp = "A10"
    period = "p240"

    data_dir = get_filepath(base_path, driver, period, post_amp, tube_r, exp_fac)    

    def path_join(filename):
        return os.path.join(data_dir, filename)
    
    cs_line = np.load(path_join("LineFlux_%s_%s_%s_cs.npy"%(driver,period,post_amp))).T
    va_line = np.load(path_join("LineFlux_%s_%s_%s_va.npy"%(driver,period,post_amp))).T
    
    return cs_line, va_line

In [6]:
y, data, beta_line, all_times, all_spoints = get_data(base_path, 'Sarch', 'r30', 'B0005')

In [7]:
y.shape, [x.shape for x in data]

((145,), [(145, 588), (145, 588), (145, 588)])

In [8]:
base_path = '/home/stuart/GitHub/Thesis/thesis/Chapter4/Data/'
tube_r = 'r30'
drivers = ['horiz', 'vert', 'Suni', 'Sarch', 'Slog']
exp_facs = [None, None, 'B0', 'B0005', 'B005']


for driver, exp_fac in zip(drivers, exp_facs):
    y, data, beta_line, all_times, all_spoints = get_data(base_path, driver, tube_r, exp_fac)
    va_line, cs_line = get_speeds(base_path, driver, tube_r, exp_fac)
    
    fig, axes = plot_paper1_td(data, y, beta_line, tube_r)
    overplot_speeds(axes, y, va_line, cs_line)

NameError: name 'plot_paper1_td' is not defined

Coordinate Stability Graph

In [28]:
fig1 = plt.figure()
plt.title("Coordinate Stability")
plt.plot(all_times, all_spoints[:,:,2]) # Plot the variation in x coordinate.
plt.ylabel("Z-coordinate")
plt.xlabel("Time")
plt.axis([0,max(all_times),0, 128])

[0, 600.03848050060856, 0, 128]

In [33]:
#==============================================================================
# Plot Time-Distance Graphs in a loop for subplots
#==============================================================================
def plot_paper1_td(data, y, beta_line, tube_r, fig_size=None):
    
    xxlim = -150
    x = all_times[:xxlim]
    lim = lambda vel: np.max([vel.max(),np.abs(vel.min())])
    def lim(vel):
        scope = np.median(vel) + 4 * np.std(np.array(vel))
        return scope
    def betaswap(b,pos):
        return "$%3.0f$"%(1./b)

    #cmap = 'gist_yarg'
    cmap = 'RdBu_r'
    
    fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=fig_size)
    #plt.subplots_adjust(left=0.1,right=0.97,top=0.94,bottom=0.10)

    if tube_r == "r30":
        manual_locations = [(75, 1.4), (25, 1.0), (50, 0.2), (25, 0.01)]
    else:
        manual_locations = False


    colorbars = []
    for i in range(0,3):

        im = NonUniformImage(axes[i], interpolation='nearest', extent=[x.min(),x.max(),y.min(),y.max()], rasterized=True)
        im.set_cmap(cmap)
        im.set_clim(vmax=lim(data[i][:,:xxlim]),vmin=-lim(data[i][:,:xxlim]))
        im.set_data(x,y,np.array(data[i])[::-1,:xxlim])

        axes[i].images.append(im)
        axes[i].set_xlim(x.min(),x.max())
        axes[i].set_ylim(y.min(),y.max())
        colorbars.append(plt.colorbar(im,ax=axes[i],aspect=10))
        ct = axes[i].contour(x,y,beta_line[:,:xxlim],colors=['k'],levels=[1.,1/3.,1/5.,1/7.,1/10.,1/100.])
        plt.clabel(ct,fontsize=18,inline_spacing=1, manual=manual_locations,fmt=mpl.ticker.FuncFormatter(betaswap))

    colorbars[0].ax.set_ylabel(r"V$_\parallel$ [m/s]")
    colorbars[1].ax.set_ylabel(r"V$_\perp$ [m/s]")
    colorbars[2].ax.set_ylabel(r"V$_\phi$ [m/s]")

    for cbar in colorbars:
        cbar.solids.set_edgecolor("face")

    #Labels
    axes[2].set_xlabel("Time [seconds]")
    axes[0].set_ylabel("Height [Mm]")
    axes[1].set_ylabel("Height [Mm]")
    axes[2].set_ylabel("Height [Mm]")

    return fig, axes


In [34]:
fig, axes = plot_paper1_td(data, y, beta_line, 'r30')

In [43]:
va_line, cs_line = get_speeds(base_path, 'horiz', 'r30', None)

In [44]:
def overplot_speeds(axes, y, va_line, cs_line):
    delta_x = np.zeros(y.shape)
    delta_x[1:] = y[1:] - y[:-1]

    delta_t_va = delta_x*1e6 / va_line[:,0]
    delta_t_cs = delta_x*1e6 / cs_line[:,0]
    delta_t_vf = delta_x*1e6 / np.sqrt(cs_line[:,0]**2 + va_line[:,0]**2)
    delta_t_vs = delta_x*1e6 / np.sqrt(cs_line[:,0]**-2 + va_line[:,0]**-2)**-1

    ti = 60
    t_va = np.cumsum(delta_t_va) + ti
    t_cs = np.cumsum(delta_t_cs) + ti
    t_vf = np.cumsum(delta_t_vf) + ti
    t_vs = np.cumsum(delta_t_vs) + ti

    for i in range(0,3):
        axes[i].plot(t_va, y, label=r"$V_A$", linewidth=2, linestyle=':', color='k')#b
        axes[i].plot(t_cs, y, label=r"$C_s$", linewidth=2, linestyle='--', color='k')#g
        axes[i].plot(t_vf, y, label=r"$V_f$", linewidth=2, linestyle='-.', color='k')#r
        axes[i].plot(t_vs, y, label=r"$V_s$", linewidth=2, linestyle='-', color='k')#c


In [45]:
overplot_speeds(axes, y, va_line, cs_line)

In [69]:
fig = plt.figure()
im = plt.imshow(data[0])
cbar = plt.colorbar()
cbar.values?